In [1]:
import joblib
import os
from numpy.linalg import inv
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['eva.pkl',
 'transformer(23_to_54).pkl',
 'ANN(energy).pkl',
 'OperationOptimModel.pkl',
 'transformer(33_to_15).pkl',
 'transformer(54_to_23).pkl',
 'ANN(tray).pkl',
 'transformer(SP162_to_Y162).pkl',
 'transformer(54_to_33).pkl',
 'ANN(58_to_sp162).pkl',
 'transformer(43_to_33).pkl',
 'SVR(4_to_23).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train.csv', 'train(energy).csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
I = joblib.load('./model/transformer(54_to_23).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

,T10,T50,T90,N+A
129,99.4,120.1,145.8,38.941


In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.048499,0.729339,0.147327,3.691656,2.1721,3.30677,0.663652,8.343065,6.304675,8.838315,3.258735,8.363279,8.655928,6.761713,6.102179,5.550676,9.621553,4.914195,3.710073,1.13431,5.44392,0.609028,0.629015


# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.036467,0.072934,0.605351,1.048499,0.014587,0.147327,2.128658,0.043442,3.691656,2.145289,0.603905,0.663652,1.161481,0.663566,0.013237,5.023968,0.818123,8.343065,0.0,4.739099,3.281093,0.086559,3.258735,8.569369,8.363279,6.761713,1.448993,0.82196,2.028806,1.80242,9.37714,0.244413,5.550676,0.052584,2.948382,1.965813,0.224293,0.963217,1.423527,1.046451,0.915469,1.358327,3.170124,0.609028,0.9512,0.588191,0.179902,0.034196,0.003207,0.006627,0.0


# B2 54 to 23

In [9]:
B.W.shape

(23, 54)

# C

In [10]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1).reset_index(drop=True)
C_X = case.join(B_pred)
C_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,30.0,10.0,100.0,1.5,0.0,0.0,0.0,0.036467,0.072934,0.605351,1.048499,0.014587,0.147327,2.128658,0.043442,3.691656,2.145289,0.603905,0.663652,1.161481,0.663566,0.013237,5.023968,0.818123,8.343065,0.0,4.739099,3.281093,0.086559,3.258735,8.569369,8.363279,6.761713,1.448993,0.82196,2.028806,1.80242,9.37714,0.244413,5.550676,0.052584,2.948382,1.965813,0.224293,0.963217,1.423527,1.046451,0.915469,1.358327,3.170124,0.609028,0.9512,0.588191,0.179902,0.034196,0.003207,0.006627,0.0


In [11]:
C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

0    54.0
dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

# D

In [12]:
D_pred = D(B_pred,C_pred)
print(D_pred.sum(axis=1))
D_pred

0    300.0
dtype: float64


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# E

In [13]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
189-14,20.0,5.0,100.0,1.5,0.0,0.0,0.0,0.018,0.036,0.296997,0.724993,0.007,0.260997,3.435966,0.069999,4.140959,3.691963,0.592994,1.04699,1.99898,0.651994,0.013,4.935951,1.140989,5.846942,0.0,6.607935,4.574955,0.085999,3.905961,8.498916,6.292938,7.775923,1.284987,0.728993,1.798982,1.597984,7.891922,0.205998,4.080959,0.045,2.710973,1.807982,0.193998,0.831992,1.228988,0.903991,0.725993,1.076989,2.512975,0.677993,1.516985,1.150988,0.286997,0.066999,0.005,0.013,0.0


In [14]:
E_pred = E.predict(E_X)
E_pred

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
0,2.42319,4.688488,0.744401,0.695318,0.749971,0.771966


# F

In [15]:
F_X = D_pred[col_names['xhc']]
F_X

,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product Properties_P-Xylene_wt%,Heart Cut Product Properties_M-Xylene_wt%,Heart Cut Product Properties_O-Xylene_wt%,Heart Cut Product Properties_2-Methyloctane_wt%,Heart Cut Product Properties_1-Nonene_wt%,Heart Cut Product Properties_n-Nonane_wt%,Heart Cut Product Properties_i-Propylbenzene_wt%,"Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Heart Cut Product Properties_n-Propylcyclohexane_wt%,Heart Cut Product Properties_n-Propylbenzene_wt%,"Heart Cut Product Properties_1,3,5-Trimethylbenzene_wt%",Heart Cut Product Properties_1-Methyl-2-ethylbenzene_wt%,"Heart Cut Product Properties_1,2,3-Trimethylbenzene_wt%","Heart Cut Product Properties_2,4-Dimethyloctane_wt%","Heart Cut Product Properties_2,2,3,3-Tetramethylhexane_wt%",Heart Cut Product Properties_3-Methylnonane_wt%,Heart Cut Product Properties_tert-Butylcyclohexane_wt%,Heart Cut Product Properties_n-Decane_wt%,"Heart Cut Product Properties_1,2-Diethylbenzene_wt%",Heart Cut Product Properties_n-Undecane_wt%,Heart Cut Product Properties_n-Pentylbenzene_wt%,Heart Cut Product Properties_n-Dodecane_wt%,Heart Cut Product Properties_n-Hexylbenzene_wt%,Heart Cut Product Properties_n-Tridecane_wt%
0,0.0,0.0,0.0,0.000591,0.003124,0.038386,0.086357,0.00118,0.016012,0.390057,0.012551,0.833661,0.887182,0.370653,0.304915,0.7401,0.493778,0.012957,5.634377,0.950885,10.071226,0.0,5.743464,3.977187,0.104933,3.950012,10.389658,10.136968,8.171614,1.752223,0.992226,2.448042,2.124127,10.374332,0.240074,4.907532,0.046658,3.370238,1.620845,0.183181,0.745858,1.168302,0.772282,0.760385,1.076863,2.390663,0.452378,0.690617,0.411125,0.121172,0.022689,0.002087,0.00427,0.0


In [16]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0    100.0
dtype: float64


,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.000591,0.00125,0.001718,0.000156,0.086357,0.039566,0.016012,0.833661,0.402609,1.627281,0.304915,10.071226,6.511765,10.671536,3.950012,10.136968,10.494591,8.171614,1.752223,0.992226,2.448042,2.124127,4.907532,10.614407,4.991083,2.916282,0.813876,4.227912,0.452378,0.438083


# G

In [17]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0    100.0
dtype: float64


,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10
0,0.0,0.0,0.0,0.000591,0.002968,59.140627,25.929904,0.304915,3.950012,1.752223,0.992226,2.448042,2.124127,2.916282,0.438083


# H

In [18]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

,"N+2A in Naphtha Feed, wt%","Chlorine in Fresh Catalyst, wt%","Carbon in Spent Catalyst, %","C6P- in Naphtha Feed, wt%","3R211 Inlet Tem, oC","3R212 Inlet Temp, oC","3R213 Inlet Temp, oC","3R214 Inlet Temp, oC",Feed Hydrogen,Feed Methane,Feed Ethane,Feed Propane,Feed n-Butane,Feed i-Butane,Feed C4 Naphthenes,Feed n-Pentane,Feed C5 i-Paraffins,Feed C5 Naphthenes,Feed n-Hexane,Feed C6 i-Paraffins,Feed C6 Naphthenes,Feed Benzene,Feed n-Heptane,Feed C7 i-Paraffins,Feed C7 Naphthenes,Feed Toluene,Feed n-Octane,Feed C8 i-Paraffins,Feed C8 Naphthenes,Feed Ethylbenzene,Feed P-Xylene,Feed M-Xylene,Feed O-Xylene,Feed n-Nonane,Feed C9 i-Paraffins,Feed C9 Naphthenes,Feed C9 Aromtics,Feed C10+ n-Paraffins,Feed C10+ i-Paraffins,Feed C10+ Naphthenes,Feed C10+ Aromtics,"Product Separator Pressure, kg/cm2_g",H2/HC Ratio
0,0.198991,0.588054,0.711188,0.724144,0.975888,0.432792,0.167189,0.661523,0.946894,0.861566,0.854109,0.249931,0.663884,0.25715,0.956112,0.825599,0.36355,0.178984,0.969122,0.502506,0.89132,0.463984,0.203843,0.164484,0.405947,0.978757,0.17713,0.032428,0.291878,0.672656,0.236268,0.469946,0.357507,0.877562,0.657335,0.319787,0.449911,0.573889,0.8554,0.24109,0.810514,0.082845,0.716695


In [19]:
H.predict(x)

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,0.477036,0.168151,0.962118,0.400819,0.451744,0.572794,0.0,0.260172,0.409088,0.00203,0.130732,0.322536,0.012828,0.356027,0.379025,1.330249,0.094559,2.882699,0.131415,0.340748,0.110612,0.735014,0.461167,3.102049,1.139551,0.000887,0.066516,0.000326,2.173062,0.002401,0.023437,0.003937,0.257314


# I

In [20]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.048499,0.729339,0.147327,3.691656,2.1721,3.30677,0.663652,8.343065,6.304675,8.838315,3.258735,8.363279,8.655928,6.761713,6.102179,5.550676,9.621553,4.914195,3.710073,1.13431,5.44392,0.609028,0.629015


In [21]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.036467,0.072934,0.605351,1.048499,0.014587,0.147327,2.128658,0.043442,3.691656,2.145289,0.603905,0.663652,1.161481,0.663566,0.013237,5.023968,0.818123,8.343065,0.0,4.739099,3.281093,0.086559,3.258735,8.569369,8.363279,6.761713,1.448993,0.82196,2.028806,1.80242,9.37714,0.244413,5.550676,0.052584,2.948382,1.965813,0.224293,0.963217,1.423527,1.046451,0.915469,1.358327,3.170124,0.609028,0.9512,0.588191,0.179902,0.034196,0.003207,0.006627,0.0


In [22]:
I(B_pred)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.048499,0.729339,0.147327,3.691656,2.1721,3.30677,0.663652,8.343065,6.304675,8.838315,3.258735,8.363279,8.655928,6.761713,6.102179,5.550676,9.621553,4.914195,3.710073,1.13431,5.44392,0.609028,0.629015


# 把全部模組整合到一塊

In [23]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_sp162).pkl')
        self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.I = joblib.load('./model/transformer(54_to_23).pkl')
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')
    
    def __call__(self,X):
        x4,case4 = X.iloc[:,:4],X.iloc[:,4:]
        # model forward
        self.y23 = self.A.predict(x4) #percent sum = 100
        self.Xna = self.B(self.y23) #percent sum = 100
        self.sp162 = self.C.predict(case4.join(self.Xna)) #percent sum = 1 * 54component = 54
        self.y162 = self.D(self.Xna,self.sp162) #percent sum = 100*3(le,hc,he) = 300
        self.xhc = self.y162[self.col_names['xhc']] #percent sum = 100
        self.xhc33_p = self.F(self.xhc) # percent 
        
        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:,:2]
        self.density = E_pred.iloc[:,2:]
        
        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]
        
        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:,2].values
        
        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel())/
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel())/
                                       (self.Xna.values@self.shc.values.ravel()))
        
        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton
        
        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*200000/self.fna_ton
        self.fhc_ton = self.fhc_ton*200000/self.fna_ton
        self.fhe_ton = self.fhe_ton*200000/self.fna_ton
        
        # add columns name
        self.fle_ton = pd.Series(self.fle_ton,name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(self.fhe_ton,name=self.col_names['Rate_ton'][3])
        
        # scale duty
        self.duty *= 200/self.fna_ton[0]
        
        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        
        # xhe33 = H(someting + xhc33 + something) 
        xhc23 = self.I(self.xhc)
        C6Pm = xhc23[['C5NP','C5IP','C6IP','C6NP']].sum(axis=1)[0]
        
        NpA = xhc23[['C5N','C6N','C7N','C8N','C9N','C10N']].sum(axis=1)[0]+\
              xhc23[['C6A','C7A','C8A','C9A','C10A']].sum(axis=1)[0]*2
        
        H_input = [NpA,0.942,3.78,C6Pm,517,517,517,515]+self.xhc33.values.ravel().tolist()+[4.798,1.44]
        H_input = np.array([H_input])
        H_input = pd.DataFrame(H_input,columns=self.H.x_col)
        
        self.重組33 = self.H.predict(H_input)
        
        # final output
        self.重組15 = self.G(self.重組33)
        
        # raname and combine return to user
        self.predict = self.重組15.join(self.fle_ton).join(self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.重組33

In [24]:
eva = EVA()

# save eva

In [25]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [26]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [27]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

,T10,T50,T90,N+A,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%
0,99.4,120.1,145.8,38.941,30.0,10.0,100.0,1.5


# get eva result

In [28]:
eva(X)

In [29]:
eva.predict

,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10,Light End Product Properties_Light End Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr
0,429229.8125,151300.03125,865698.1875,360650.65625,507991.164062,3.747139e+06,244025.718323,320347.375,2593807.0,661353.6875,414951.125,1929344.375,1025350.25,3005107.75,231527.125,20121.075821,15000.672368,5.201888,9.123536


In [30]:
eva.xhc33

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,97.489472,206.027323,283.287569,25.753415,14238.35196,6523.567765,2639.985295,137452.62653,66381.453246,268303.284927,50273.875221,1.660526e+06,1.073648e+06,1.759504e+06,651271.052601,1.671366e+06,1.730330e+06,1.347321e+06,288903.417974,163596.558403,403628.8693,350222.273204,809145.333813,1.750085e+06,822921.00366,480831.446839,134190.525335,697090.680032,74587.297941,72230.375239


In [31]:
eva.naphtha

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.048499,0.729339,0.147327,3.691656,2.1721,3.30677,0.663652,8.343065,6.304675,8.838315,3.258735,8.363279,8.655928,6.761713,6.102179,5.550676,9.621553,4.914195,3.710073,1.13431,5.44392,0.609028,0.629015


In [32]:
eva.pre_d

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

In [33]:
eva.reform

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,429229.8125,151300.03125,865698.1875,360650.65625,406472.09375,101519.070312,0.0,616586.375,930862.6875,1826.630127,117630.203125,290212.375,11542.707031,320347.375,341040.625,1196937.0,85082.601562,2593807.0,118245.234375,68056.3125,99526.867188,661353.6875,414951.125,1929344.375,1025350.25,798.123657,3470.390137,293.224915,3005107.75,2159.975586,61139.957031,45753.6875,231527.125
